時系列データというと1分間とか一定の時間毎の間隔のデータを想像すると思う。しかし、投資を分析をする際に一定時間毎のデータを見ても良い分析ができるかはわからない。実際の市場は同じ1分なり1時間でも、なんらかの情報が来た直後なら取引が多く情報も多くなるし、特になければ注文がなくて情報もスカスカになるからである。
<br>
<br>
そこで一定の出来高（注文が成立した量）や一定回数の値動き毎にサンプリングするなど、なるべく1本あたりのデータ間の情報量を均一にしようという試みが思いつく。
<br>
<br>
言い忘れてたけど一定時間毎だか出来高ごとにしても抽出したデータ（からなるテーブル）のことを「バー」と呼ぶらしい
<br>
タイムバーなら一定時間毎にサンプリングされたやつだし、ボリュームバーなら一定の出来高ごとにサンプリングされたデータのテーブルを指す
<br>
<br>
なおこの情報が書いてあった書籍には↑みたいな内容が書いてあったが、ファイナンス系の時系列データ分析が専門の教員に聞いたら「あーそんなのもあるらしいね。どうなんだろうね？」くらいの反応だったので本当に効果があるかは知らん

In [7]:
#ライブラリのインポート

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time, datetime, math
import matplotlib.pyplot as plt
import collections

# ボリュームバー



ここではボリュームバー（一定の出来高ごとのサンプリング）を作るがその前に言っておかなければならないことがある。それは"ティック”の存在である。
<br>
ティックとは約定（注文が成立すること）した際のデータのことである。約定した価格と注文量、注文が成立した時間、買いか売りか、注文のIDとかが含まれている。
<br>
これを加工してタイムバーなりボリュームバーを作るわけである。

In [2]:
#ティックデータのロード
table = pd.read_csv("/content/drive/My Drive/cryptocurrency/data/tick_ETH.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
table

,quantity,price,taker_side,created_at,iso_time
id,,,,,
5449226,40.000000,1200.0,buy,1483308241,2017-01-01T22:04:01Z
5449231,1.800000,1200.0,buy,1483308277,2017-01-01T22:04:37Z
5449232,0.200000,1250.0,buy,1483308277,2017-01-01T22:04:37Z
5482479,19.800000,1250.0,buy,1483438258,2017-01-03T10:10:58Z
5482480,4.200000,1290.0,buy,1483438258,2017-01-03T10:10:58Z
...,...,...,...,...,...
338926783,0.640640,26130.0,buy,1595415398,2020-07-22T10:56:38Z
338926789,0.550240,26130.0,sell,1595415399,2020-07-22T10:56:39Z
338926791,0.550240,26130.0,buy,1595415399,2020-07-22T10:56:39Z


In [9]:
max(table["quantity"])

2276.0

In [8]:
#どのくらいの注文数(threshold)でボリュームバーを作るのか指定、一定の注文数を超える毎にindexを記録する

threshold = 2300
sum_=np.cumsum(table["quantity"])
th_idx = []

cache = 0
check_point=-1

while check_point!=cache:
  check_point = cache
  for idx, s in enumerate(sum_):
    if s - cache>threshold:
      th_idx.append(idx)
      cache = s

In [10]:
#さっき決めたindexに基づいてボリュームバーを作成する
#なおティックデータから作れる特徴量はここで作っておく

bar = []
cache = 0
timestamps=[]
before_close=1

for idx in tqdm(th_idx):
  #該当範囲だけ切り取る
  inst = table.iloc[cache:idx+1, :]

  #OHLC(始値、高値、安値、終値)を求める
  open_=inst["price"].iloc[0]
  high=max(inst["price"])
  low=min(inst["price"])
  close=inst["price"].iloc[-1]

  #バーに含まれる最後のティックが作成された時刻、これをボリュームバーのindexにする
  timestamp=inst["iso_time"].iloc[-1]

  #バー内に含まれる売り買い毎のの注文数
  num_buy = list(inst["taker_side"]).count("buy")
  num_sell = list(inst["taker_side"]).count("sell")

  #バー内に含まれるティック数
  num_tick = idx + 1 - cache

  #Hasbrouckのλ（取引コストを近似するものらしい、考えてはいけない）
  nume = np.log(close) - np.log(before_close)
  deno = 0
  side_=list(inst["taker_side"])
  price_=list(inst["price"])
  quan_=list(inst["quantity"])
  for lambda_idx in range(1, inst.shape[0]):
    if side_[lambda_idx]=="buy":
      b=1
    else:
      b=-1
    deno+=b*np.sqrt(price_[lambda_idx]*quan_[lambda_idx])
  h_lambda = nume / deno
  before_close = close


  #ラウンドサイズ(キリのいいサイズ)の取引数を計測。ティックに含まれる注文の最小サイズが1e-8くらいなのでこれを目安に1000のくらいにでも飛ばす
  #(とはいえ本来だと0.00１あたりが最小の注文数のはずなので何かがおかしい？)
  #その後キリいいかmod1000で判断(実際1e-5~1e-8はそんなにない)
  round_size=inst["quantity"][inst["quantity"]*10e+8%1000==0].shape[0]

  #バー内に含まれる売り買いごとのボリュームの合計
  volume_buy=sum(inst["quantity"][inst["taker_side"]=="buy"])
  volume_sell=sum(inst["quantity"][inst["taker_side"]=="sell"])
  
  #バー内の売り買いごとの最大ボリューム
  try:
      max_volume_buy=max(inst["quantity"][inst["taker_side"]=="buy"])
  except:
      max_volume_buy=0
  try:
      max_volume_sell=max(inst["quantity"][inst["taker_side"]=="sell"])
  except:
      max_volume_sell=0
  

  #データの追加
  bar.append([open_, high, low, close, timestamp, num_tick, num_buy, num_sell,\
                        volume_buy, volume_sell, max_volume_buy, max_volume_sell, h_lambda, round_size])
  timestamps.append(timestamp)

  #キャッシュの更新、次のやつのスタートになるように設定
  cache=idx+1

volume_bar =pd.DataFrame(bar, index=timestamps,\
                                 columns=["open", "high", "low", "close", "timestamp", "num_tick", "num_buy", "num_sell", "volume_buy", "volume_sell",\
                                          "max_volume_buy", "max_volume_sell", "Hasbrouck_lambda", "round_size"])
volume_bar.index=pd.to_datetime(volume_bar.index, utc=True)
volume_bar.drop("timestamp", axis=1, inplace=True)

In [11]:
volume_bar

,open,high,low,close,num_tick,num_buy,num_sell,volume_buy,volume_sell,max_volume_buy,max_volume_sell,Hasbrouck_lambda,round_size
2017-01-27 03:43:32+00:00,1200.00,1450.00,999.00,1401.40,102,57,45,1795.824835,508.675000,422.000000,181.800000,1.563166e-03,94
2017-02-09 01:47:26+00:00,1401.40,1401.40,1190.80,1320.00,64,25,39,812.414378,1971.606171,575.416046,1043.953154,2.514745e-05,48
2017-02-18 10:41:28+00:00,1320.00,1649.90,1191.20,1341.35,91,62,29,2088.965302,326.892800,422.000000,40.000000,2.366436e-06,80
2017-02-22 07:32:01+00:00,1488.05,1686.31,1300.00,1678.32,43,31,12,1172.928000,1513.463600,392.284546,400.000000,3.250669e-04,29
2017-02-25 21:47:29+00:00,1439.76,1899.99,1439.76,1524.74,33,26,7,1617.531054,697.224946,460.000000,399.991600,-1.986988e-05,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-21 07:35:40+00:00,25337.00,25728.00,25301.00,25713.00,1580,855,725,1478.195140,825.759259,80.000000,149.202412,4.942525e-07,1246
2020-07-21 12:17:23+00:00,25711.00,26271.00,25675.00,26271.00,1477,679,798,1151.286242,1181.593547,60.000000,47.398523,-1.135661e-06,1210
2020-07-21 16:04:07+00:00,26290.00,26369.00,25985.00,26064.00,1408,816,592,1526.827535,773.308577,60.000000,60.000000,-1.816322e-07,1139
2020-07-22 02:32:18+00:00,26072.00,26369.00,26027.00,26102.00,1890,969,921,1205.180755,1099.830789,80.000000,89.600000,1.127924e-07,1585


In [ ]:
volume_bar.to_csv("/content/drive/My Drive/cryptocurrency/data/volume_bar_ETH.csv")

# タイムバー

タイムバーのが嬉しいこともあるかもしれないので作っておく

In [12]:
table=table.set_index("iso_time")
table.index=pd.to_datetime(table.index)

In [13]:
#どの感覚でサンプリングするか指定して実行、細かいことはpandasのdocument見て
rule=datetime.timedelta(minutes=1)
time_bar=table["price"].resample(rule=rule).ohlc()

#欠損値の穴埋め、まぁ前のを採用すればいいと思う
time_bar["open"].fillna(method="ffill", inplace=True)

In [14]:
time_bar

,open,high,low,close
iso_time,,,,
2017-01-01 22:04:00+00:00,1200.0,1250.0,1200.0,1250.0
2017-01-01 22:05:00+00:00,1200.0,NaN,NaN,NaN
2017-01-01 22:06:00+00:00,1200.0,NaN,NaN,NaN
2017-01-01 22:07:00+00:00,1200.0,NaN,NaN,NaN
2017-01-01 22:08:00+00:00,1200.0,NaN,NaN,NaN
...,...,...,...,...
2020-07-22 10:54:00+00:00,26130.0,NaN,NaN,NaN
2020-07-22 10:55:00+00:00,26130.0,NaN,NaN,NaN
2020-07-22 10:56:00+00:00,26130.0,26130.0,26130.0,26130.0


In [ ]:
#欠損値における高値、安値、終値を始値に統一する
#クッソ遅いけどコードの書き方が思いつかなかったので保留
#そういえばpandasのilocよりも、seriesをlistにしてから参照した方が爆速な気がする

open_=list(time_bar["open"])
for i in tqdm(range(time_bar.shape[0])):
  if time_bar.iloc[i].isnull().sum()>0:
    time_bar.iloc[i, 1:]=open_[i]

In [ ]:
time_bar.to_csv("/content/drive/My Drive/cryptocurrency/data/time_bar_ETH.csv")